### 🧭 ML Pipeline Overview – Module 3

| **Stage**                  | **Description**                                        | **Output**                    |
|---------------------------|--------------------------------------------------------|-------------------------------|
| 1. Download the data      | Ingest from S3, GCS, or URL                            | Raw data file(s)              |
| 2. Transforming data      | Filter, clean, drop columns, remove outliers           | Cleaned DataFrame             |
| 3. Preparing data for ML  | Feature engineering, vectorization, label split        | `X_train`, `y_train`, `X_val`, `y_val` |
| 4. Hyperparameter tuning  | Hyperopt, Optuna, or GridSearch                        | Best model parameters         |
| 5. Train the final model  | Retrain using best parameters                          | Fitted model                  |
| 6. Model registry         | Log final model to MLflow Model Registry               | Versioned model entry         |

In [1]:
!python -V


Python 3.9.21


# 📦 Imports

In [ ]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
import mlflow
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

# 📄 Read CSV with preprocessing

In [ ]:

def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

# 📊 Load Data

In [ ]:

df_train = read_dataframe('./data/green_tripdata_2021-01.csv')
df_val = read_dataframe('./data/green_tripdata_2021-02.csv')

# 🧪 Feature Engineering

In [ ]:

df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

# 📈 Train & Evaluate Linear Regression

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
print("Linear Reg RMSE:", mean_squared_error(y_val, y_pred, squared=False))


# 💾 Save model

In [ ]:

with open("models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)


# 📋 Track with MLflow (Lasso Example)

In [ ]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

with mlflow.start_run():
    mlflow.set_tag("developer", "cristian")
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    lasso = Lasso(alpha)
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")


# 🚀 XGBoost + Hyperopt Tuning

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)

        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

        return {"loss": rmse, "status": STATUS_OK}

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

# MLflow Logging for Multiple Models

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error
import mlflow
import pickle

regressors = [
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesRegressor,
    LinearSVR
]

# Save preprocessor
with open("models/preprocessor.b", "wb") as f_out:
    pickle.dump(dv, f_out)

for model_class in regressors:
    model_name = model_class.__name__

    with mlflow.start_run(run_name=model_name):
        mlflow.set_tag("model_type", model_name)
        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        model = model_class()
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

### 🧱 Modular ML Pipeline as Python Functions

A clean function-based structure for building a machine learning workflow, which can later be orchestrated using tools like Prefect or Airflow.

In [ ]:
def download_data(year, month):
    ...
    return df

def prepare_data(df):
    ...
    return df

def feature_engineering(df):
    ...
    return X, y

def find_best_model(X, y):
    ...
    return params

def train_model(X, y, params):
    ...
    return model

def main():
    df = download_data(2023, 1)
    df = prepare_data(df)
    X, y = feature_engineering(df)
    model_params = find_best_model(X, y)
    model = train_model(X, y, model_params)